# Run Pharaglow 

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
# image io and analysis
import json
import pims
import trackpy as tp

# plotting
import matplotlib  as mpl 
import matplotlib.pyplot as plt 

#our packages
from pharaglow import tracking, run, features

### Load data and create binary masks 

In [ ]:
%%time
# io
fname = "/media/scholz_la/hd2/maxi/Substrate stiffness/MV0021/*.tiff"
parameterfile = "/media/scholz_la/hd2/maxi/Substrate stiffness/pharaglow_parameters_2x2.txt"
outfile = "/media/scholz_la/hd2/maxi/Substrate stiffness/MV0021_{}_{}.json"
lawnfile = None#'/media/scholz_la/hd2/Nicolina/Nicolina/NZ0014_lawn.bmp'

print('Starting pharaglow analysis...')
rawframes = pims.open(fname)
rawframes = rawframes

print('Loading parameters from {}'.format(parameterfile.split('/')[-2:]))
with open(parameterfile) as f:
    param = json.load(f)

if lawnfile is not None:
    print('open and binarize lawn file')
    lawn = pims.open(lawnfile)[0]
    binLawn = features.findLawn(lawn)

In [ ]:
plt.figure(figsize=(15,8))
plt.imshow(binLawn)

In [ ]:
%%time
# detecting objects
print('Binarizing images')
masks = tracking.calculateMask(rawframes, minSize = param['minSize'], bgWindow = param['bgWindow']
                               , thresholdWindow = param['thresholdWindow'], smooth =  param['smooth'],
                               subtract =  param['subtract'])


### Make sure the thesholding worked otherwise change parameters

In [ ]:
t = 7000
plt.figure(figsize=(18,11))
# plt.subplot(121)
plt.imshow(rawframes[t])
# plt.subplot(122)
plt.imshow(masks[t])#[:,2500:])
print(np.sum(masks[t]))#[:,2500:]))

### Detecting individual objects and tracking or use multiprocessing to speed up feature detection

In [ ]:
%%time
import time

nWorkers = 10
if nWorkers ==1:

    #masks = tracking.preprocess(rawframes, minSize = param['minSize'], threshold =None )
    print('Detecting features')
    features = tracking.runfeatureDetection(rawframes, masks, param, frameOffset = 0)
else:
    from multiprocessing import Pool
    print('Detecting features')
    def f(sl):
        a,b = sl
        #print(a,b)
        return tracking.runfeatureDetection(rawframes[a:b], masks[a:b], param, frameOffset = a)
    features = []
    L = len(rawframes)
    # create chunks of analysis based on how many workers we use
    print(L)
    chunksize = L//nWorkers//20
    #slices = np.arange(L)
    slices = zip((range(0,L, chunksize)), (range(chunksize,L+chunksize, chunksize)))
    
    p = Pool(processes = nWorkers)
    start = time.time()
    for k, res in enumerate(p.imap_unordered(f, slices)):
        features.append(res)
        if k ==10:
            print('Expected time is approx. {} s'.format(L/chunksize*(time.time()-start)/nWorkers))
        #print(p, time.time()-start)
    features = pd.concat(features)

In [ ]:

### Save the features
plt.plot(np.sort(features['area'].values))

In [ ]:
features.head(5)
features.info(memory_usage='deep')
features.to_json(outfile.format('features', 'all'), orient='split')

In [ ]:
features = pd.read_json(outfile.format('features', 'all'), orient='split', numpy = True)

In [ ]:
print('Done')
print('Linking trajectories')
#trajectories = tracking.linkParticles(features, param['searchRange'], param['minimalDuration'])
#trajectories = tracking.linkParticles(features, 50, 500);
#trajectories = tp.link_df(features, 50, memory= 30)
#trajectories = tp.filter_stubs(trajectories,100)
pred = tp.predict.NearestVelocityPredict()
trajectories = pred.link_df(features, 30, memory= 30)

In [ ]:
print(trajectories['particle'].nunique())
trajectories = tp.filter_stubs(trajectories,300)
print(trajectories['particle'].nunique())

### Extract lawn info

In [ ]:
%%time

def inside(x,y,binLawn):
    return binLawn[int(y), int(x)]

if lawnfile is not None:
    trajectories['inside'] = trajectories.apply(\
        lambda row: pd.Series(inside(row['x'], row['y'], binLawn)), axis=1)

### Show resulting trajectories

In [ ]:
plt.figure(figsize=(11,11))
tp.plot_traj(trajectories)#, superimpose=1-masks[0]);

In [ ]:
trajectories.to_json(outfile.format('trajectories', 'all'), orient='split')

In [ ]:
from multiprocessing import Pool
def parallelize_dataframe(df, func, params, n_cores):
    df_split = np.array_split(df, n_cores)
    print([len(d) for d in df_split])
    pool = Pool(n_cores)
    df = pd.concat(pool.starmap(func, zip(df_split, np.repeat(params, len(df_split)))))
    pool.close()
    pool.join()
    return df

### run the whole pharaglow feature extraction

In [ ]:
# # read data
# trajectories = pd.read_json(outfile.format('trajectories', 'all'), orient='split', numpy = True)

In [ ]:
# %%time
# print('Extracting pharynx data')
# trajectories = parallelize_dataframe(trajectories, run.runPharaglowOnStack, n_cores = 20, params = param)
# print('Done tracking. Successfully tracked {} frames with {} trajectorie(s).'.format(len(rawframes), trajectories['particle'].nunique()))

### Save data as hdf5 format (every trajectory in a file)

In [ ]:
%%time
#trajectories.info(memory_usage='deep')

for particle_index in trajectories['particle'].unique():
    tmp = parallelize_dataframe(trajectories[trajectories.loc[:,'particle'] == particle_index], run.runPharaglowOnStack, n_cores = 10, params = param)
    tmp.to_json(outfile.format('results', particle_index), orient='split')